<a href="https://colab.research.google.com/github/chagas98/BioinfoWikidatascripts/blob/main/wiki_bioinfo_brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Bibliotecas**



In [ ]:
import numpy as np    
import pandas as pd
from google.colab import data_table
from time import sleep, ctime, gmtime, strftime

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()



### **Dados de Entrada**

Link dos arquivos .csv disponíveis nos Dados Abertos da CAPES. Classificados em: Programas (prog), Discentes (stud) e Docentes (prof) (dados 2017-2021)



In [ ]:
discentes = {2005: "https://dadosabertos.capes.gov.br/dataset/e4847dfa-9fce-4962-a03f-c45c69b20775/resource/311ad2fd-6c39-429c-a882-042578add42a/download/br-capes-colsucup-discentes-2005-2021-03-01.csv",
             2020: "https://dadosabertos.capes.gov.br/dataset/b7003093-4fab-4b88-b0fa-b7d8df0bcb77/resource/4e30fa75-42a2-428a-bd98-0949deb3c562/download/br-capes-colsucup-discentes-2020-2021-11-10.csv"}
             

In [ ]:
discentes = {2005: "https://dadosabertos.capes.gov.br/dataset/e4847dfa-9fce-4962-a03f-c45c69b20775/resource/311ad2fd-6c39-429c-a882-042578add42a/download/br-capes-colsucup-discentes-2005-2021-03-01.csv",
             2006: "https://dadosabertos.capes.gov.br/dataset/e4847dfa-9fce-4962-a03f-c45c69b20775/resource/7dfaa0b2-5b13-4a88-a567-4e4b1c76453f/download/br-capes-colsucup-discentes-2006-2021-03-01.csv",
             2007: "https://dadosabertos.capes.gov.br/dataset/e4847dfa-9fce-4962-a03f-c45c69b20775/resource/f5285048-2220-4d8d-8c96-7f48bb25233e/download/br-capes-colsucup-discentes-2007-2021-03-01.csv",
             2008: "https://dadosabertos.capes.gov.br/dataset/e4847dfa-9fce-4962-a03f-c45c69b20775/resource/c34dc5f8-0c08-41c5-9f7e-f9ffa8f09cd3/download/br-capes-colsucup-discentes-2008-2021-03-01.csv",
             2009: "https://dadosabertos.capes.gov.br/dataset/e4847dfa-9fce-4962-a03f-c45c69b20775/resource/8aa637c9-33c0-4ae7-a7bb-e49cc83dc66c/download/br-capes-colsucup-discentes-2009-2021-03-01.csv",
             2010: "https://dadosabertos.capes.gov.br/dataset/e4847dfa-9fce-4962-a03f-c45c69b20775/resource/8921810c-6e41-4ee9-adfc-895b9cc08bfa/download/br-capes-colsucup-discentes-2010-2021-03-01.csv",
             2011: "https://dadosabertos.capes.gov.br/dataset/e4847dfa-9fce-4962-a03f-c45c69b20775/resource/73dff194-b8a8-49cc-a4d7-63ecae069e9b/download/br-capes-colsucup-discentes-2011-2021-03-01.csv",
             2012: "https://dadosabertos.capes.gov.br/dataset/e4847dfa-9fce-4962-a03f-c45c69b20775/resource/273262e4-867e-428a-816e-f58623c0e225/download/br-capes-colsucup-discentes-2012-2021-03-01.csv",
             2013: "https://dadosabertos.capes.gov.br/dataset/dc2568b7-20b0-4d92-980d-dcf2485b5517/resource/c85de819-6972-434f-b386-002d201596dd/download/br-capes-colsucup-discentes-2013-2021-03-01.csv",
             2014: "https://dadosabertos.capes.gov.br/dataset/dc2568b7-20b0-4d92-980d-dcf2485b5517/resource/4301273c-b10a-4b8f-8421-e54877565648/download/br-capes-colsucup-discentes-2014-2021-03-01.csv",
             2015: "https://dadosabertos.capes.gov.br/dataset/dc2568b7-20b0-4d92-980d-dcf2485b5517/resource/44cd2352-b06a-4624-a846-e09534d1e9f0/download/br-capes-colsucup-discentes-2015-2021-03-01.csv",
             2016: "https://dadosabertos.capes.gov.br/dataset/dc2568b7-20b0-4d92-980d-dcf2485b5517/resource/37875416-2b0a-4249-a472-060b6b2b6c72/download/br-capes-colsucup-discentes-2016-2021-03-01.csv",
             2017: "https://dadosabertos.capes.gov.br/dataset/b7003093-4fab-4b88-b0fa-b7d8df0bcb77/resource/e9f13006-d0d3-4869-9790-1a8f5d31cf0c/download/br-capes-colsucup-discentes-2017-2021-11-10.csv",
             2018: "https://dadosabertos.capes.gov.br/dataset/b7003093-4fab-4b88-b0fa-b7d8df0bcb77/resource/84290e6a-0a56-4372-aec7-7a188ff01e27/download/br-capes-colsucup-discentes-2018-2021-11-10.csv",
             2019: "https://dadosabertos.capes.gov.br/dataset/b7003093-4fab-4b88-b0fa-b7d8df0bcb77/resource/967f83b7-3997-4cfc-8273-b9aeb8fdb9e2/download/br-capes-colsucup-discentes-2019-2021-11-10.csv",
             2020: "https://dadosabertos.capes.gov.br/dataset/b7003093-4fab-4b88-b0fa-b7d8df0bcb77/resource/4e30fa75-42a2-428a-bd98-0949deb3c562/download/br-capes-colsucup-discentes-2020-2021-11-10.csv"}


In [102]:
discentes = {2020: "https://dadosabertos.capes.gov.br/dataset/b7003093-4fab-4b88-b0fa-b7d8df0bcb77/resource/4e30fa75-42a2-428a-bd98-0949deb3c562/download/br-capes-colsucup-discentes-2020-2021-11-10.csv"}


In [ ]:
csv_links = {"prog": "https://dadosabertos.capes.gov.br/dataset/903b4215-ea91-4927-8975-d1484891374f/resource/9835dd45-d4e7-4b1f-b550-eb9b049bacac/download/br-capes-colsucup-prog-2017-2021-11-10.csv",
"stud": "https://dadosabertos.capes.gov.br/dataset/b7003093-4fab-4b88-b0fa-b7d8df0bcb77/resource/e9f13006-d0d3-4869-9790-1a8f5d31cf0c/download/br-capes-colsucup-discentes-2017-2021-11-10.csv",
"prof": "https://dadosabertos.capes.gov.br/dataset/57f86b23-e751-4834-8537-e9d33bd608b6/resource/72ab509c-4ca8-452c-9768-b591194f96e1/download/br-capes-colsucup-docente-2020-2021-11-10.csv"}



In [ ]:
data_table.enable_dataframe_formatter()


#### **Google Sheets**

Rodar apenas uma vez, cria uma tabela para armazenar informações extraídas.

In [24]:
gc = gspread.authorize(creds)

spreadsheet_name = "HackWikiBioinfoBrasil_Resultados"
sh = gc.create(spreadsheet_name)

sh = gc.open(spreadsheet_name)
print("https://docs.google.com/spreadsheets/d/%s" % sh.id)

https://docs.google.com/spreadsheets/d/1OJDcP3hF_-GdJ1fInfaR1nY3hAo9YI6VM2qklcmWWv0


###**Programas Bioinfo**

Extração de informações a partir dos programas de bioinfo cadastrados pela CAPES.

**Seleção de Informações**


*   Universidade
*   Código Programa
*   Cidade
*   Estado
*   Região

In [ ]:
dataprog = pd.read_csv(csv_links.get("prog"), sep = ";", encoding = "ISO-8859-1")

bioinfo_prog = dataprog[dataprog["NM_PROGRAMA_IES"].str.contains("BIOINFO")]

In [ ]:
select_info = ["NM_PROGRAMA_IES", "NM_PROGRAMA_IDIOMA","NM_ENTIDADE_ENSINO", "CD_PROGRAMA_IES", "NM_MUNICIPIO_PROGRAMA_IES", "SG_UF_PROGRAMA", "NM_REGIAO"]
bioinfo_progsel = bioinfo_prog.loc[:, select_info]

bioinfo_progsel["Name"] =  bioinfo_progsel["NM_PROGRAMA_IDIOMA"] + " (" + bioinfo_progsel["NM_ENTIDADE_ENSINO"] + ")"

bioinfo_progsel


,NM_PROGRAMA_IES,NM_PROGRAMA_IDIOMA,NM_ENTIDADE_ENSINO,CD_PROGRAMA_IES,NM_MUNICIPIO_PROGRAMA_IES,SG_UF_PROGRAMA,NM_REGIAO,Name
522,BIOINFORMÁTICA,BIOINFORMATICS,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,23001011170P8,NATAL,RN,NORDESTE,BIOINFORMATICS (UNIVERSIDADE FEDERAL DO RIO GR...
1760,BIOINFORMÁTICA,POST GRADUATE PROGRAM IN BIOINFORMATICS,UNIVERSIDADE FEDERAL DE MINAS GERAIS,32001010068P4,BELO HORIZONTE,MG,SUDESTE,POST GRADUATE PROGRAM IN BIOINFORMATICS (UNIVE...
2214,BIOINFORMÁTICA,INTER-DEPARTMENTAL GRADUATE PROGRAM IN BIOINFO...,UNIVERSIDADE DE SÃO PAULO,33002010188P9,SÃO PAULO,SP,SUDESTE,INTER-DEPARTMENTAL GRADUATE PROGRAM IN BIOINFO...
3130,BIOINFORMÁTICA,BIOINFORMATICS,UNIVERSIDADE FEDERAL DO PARANÁ,40001016066P4,CURITIBA,PR,SUL,BIOINFORMATICS (UNIVERSIDADE FEDERAL DO PARANÁ)
3287,BIOINFORMÁTICA,BIOINFORMATICS GRADUATE PROGRAM,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ,40006018037P6,CORNÉLIO PROCÓPIO,PR,SUL,BIOINFORMATICS GRADUATE PROGRAM (UNIVERSIDADE ...


In [ ]:
df_prog = pd.DataFrame({"names": bioinfo_progsel["Name"].to_list()})

worksheet_prog = sh.sheet1
#worksheet_prog = sh.add_worksheet("Programas", rows = "100", cols = "9")

worksheet_prog.update_cell(1, 1, df_prog.columns[0])
worksheet_prog.update_cell(1, 2, "wikidata_id")
worksheet_prog.update_cell(1, 3, "blank")
worksheet_prog.update_cell(1, 4, "wikidata_description")

for i, row in df_prog.iterrows():
  worksheet_prog.update_cell(i+2, 1, row[0])
  worksheet_prog.update_cell(i+2, 2, "=WIKIDATASEARCH(A"+ str(i+2)+")")
  worksheet_prog.update_cell(1, 3, "")
  worksheet_prog.update_cell(i+2, 4, "=WIKIDATADESCRIPTIONS(B"+ str(i+2)+")")

###**Discentes Bioinfo**

Seleção dos discentes que receberam título em um programa de PG em Bioinfo.

**Seleção de Informações**

*   Universidade
*   Nacionalidade
*   Código Programa
*   Orientador(a)
*   Grau
*   Tese

In [104]:
select_info = ["AN_BASE","NM_DISCENTE","NM_ENTIDADE_ENSINO","CD_PROGRAMA_IES","NM_ORIENTADOR_PRINCIPAL", "NM_TESE_DISSERTACAO", "NM_NIVEL_TITULACAO_DISCENTE", "NM_NIVEL_TITULACAO_DISCENTE", "NM_GRAU_PROGRAMA"]

all_students = []

for year in discentes.keys():

  datastud =  pd.read_csv(discentes.get(year), sep = ";", encoding = "ISO-8859-1")
  print(year)
  bioinfo_stud = datastud[datastud["NM_PROGRAMA_IES"].str.contains("BIOINFO") & datastud["NM_SITUACAO_DISCENTE"].str.contains("TITULADO")]
  bioinfo_studsel = bioinfo_stud.reindex(columns=select_info).reset_index(drop=True)
  all_students.append(bioinfo_studsel)
  

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2020


In [109]:
datastud = pd.read_csv(discentes.get(2020), sep = ";", encoding = "ISO-8859-1")

bioinfo_stud = datastud[datastud["NM_PROGRAMA_IES"].str.contains("BIOINFO") & datastud["NM_SITUACAO_DISCENTE"].str.contains("TITULADO")]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [111]:
select_info = ["NM_DISCENTE","NM_ENTIDADE_ENSINO","DS_TIPO_NACIONALIDADE_DISCENTE","CD_PROGRAMA_IES","NM_ORIENTADOR_PRINCIPAL", "DS_GRAU_ACADEMICO_DISCENTE", "NM_TESE_DISSERTACAO"]
bioinfo_studsel = bioinfo_stud.loc[:, select_info]


bioinfo_studsel


,NM_DISCENTE,NM_ENTIDADE_ENSINO,DS_TIPO_NACIONALIDADE_DISCENTE,CD_PROGRAMA_IES,NM_ORIENTADOR_PRINCIPAL,DS_GRAU_ACADEMICO_DISCENTE,NM_TESE_DISSERTACAO
40783,RENATA LILIAN DANTAS CAVALCANTE,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,TETSU SAKAMOTO,MESTRADO,INVESTIGAÇÃO EXPLORATÓRIA DOS FATORES GENÉTICO...
41356,KATYANNA SALES BEZERRA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,UMBERTO LAINO FULCO,DOUTORADO,ESTUDO BIOQUÍMICO QUÂNTICO DE INTERAÇÕES ENTRE...
41419,EDUARDO NOGUEIRA CUNHA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,JOAO PAULO MATOS SANTOS LIMA,DOUTORADO,UM SISTEMA INTELIGENTE DE BAIXO CUSTO PARA DET...
41546,EDEN SILVA E SOUZA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,MARILENE HENNING VAINSTEIN,MESTRADO,AVALIAÇÃO DO ALVO PREDITO DA PLUMIERIDINA EM C...
41578,LUCAS CAIA DE SOUZA TAVARES,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,ADRIANO BRETANHA LOPES TORT,MESTRADO,INTERAÇÕES HIPOCAMPO-PREFRONTAIS DURANTE A TOM...
41961,DANILO LOPES MARTINS,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,JORGE ESTEFANO SANTANA DE SOUZA,MESTRADO,ANÁLISE EXPLORATÓRIA DO TRANSCRIPTOMA DO ARAPA...
42089,FREDERICO LEMOS DOS SANTOS,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,UMBERTO LAINO FULCO,DOUTORADO,PROCESSO EPIDÊMICO MEDIADO POR VETORES E PROCE...
42344,IGOR AUGUSTO BRANDAO,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,RODRIGO JULIANI SIQUEIRA DALMOLIN,MESTRADO,ABORDAGENS DA BIOLOGIA DE SISTEMAS NA INVESTIG...
155007,THIAGO DE JESUS SOUSA,UNIVERSIDADE FEDERAL DE MINAS GERAIS,BRASILEIRO,32001010068P4,VASCO ARISTON DE CARVALHO AZEVEDO,DOUTORADO,OBTENÇÃO DE GENOMAS COMPLETOS DE ALTA QUALIDAD...
155280,CARMELINA FIGUEIREDO VIEIRA LEITE,UNIVERSIDADE FEDERAL DE MINAS GERAIS,BRASILEIRO,32001010068P4,MARCOS AUGUSTO DOS SANTOS,DOUTORADO,A NEW METHOD FOR LIGAND-BASED VIRTUAL SCREENIN...


In [114]:
bioinfo_studsel["CD_PROGRAMA_IES"] = np.where((bioinfo_studsel['CD_PROGRAMA_IES'] == "23001011170P8"), "Q105627005", bioinfo_studsel["CD_PROGRAMA_IES"]) #UFRN
bioinfo_studsel["CD_PROGRAMA_IES"] = np.where((bioinfo_studsel['CD_PROGRAMA_IES'] == "32001010068P4"), "Q105481707", bioinfo_studsel["CD_PROGRAMA_IES"]) #UFMG
bioinfo_studsel["CD_PROGRAMA_IES"] = np.where((bioinfo_studsel['CD_PROGRAMA_IES'] == "33002010188P9"), "Q102292035", bioinfo_studsel["CD_PROGRAMA_IES"]) #USP
bioinfo_studsel["CD_PROGRAMA_IES"] = np.where((bioinfo_studsel['CD_PROGRAMA_IES'] == "40001016066P4"),"Q105621954", bioinfo_studsel["CD_PROGRAMA_IES"]) #UFPR
bioinfo_studsel["CD_PROGRAMA_IES"] = np.where((bioinfo_studsel['CD_PROGRAMA_IES'] == "40001016175P8") & (bioinfo_studsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE FEDERAL DO PARANÁ"), "Q105621954", bioinfo_studsel["CD_PROGRAMA_IES"]) #UFPR
bioinfo_studsel["CD_PROGRAMA_IES"] = np.where((bioinfo_studsel['CD_PROGRAMA_IES'] == "40001016175P8") & (bioinfo_studsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ (CORNÉLIO PROCÓPIO)"), "Q105627310", bioinfo_studsel["CD_PROGRAMA_IES"]) #UTFPR
bioinfo_studsel["CD_PROGRAMA_IES"] = np.where((bioinfo_studsel['CD_PROGRAMA_IES'] == "40006018037P6"), "Q105627310", bioinfo_studsel["CD_PROGRAMA_IES"]) #UTFPR


In [116]:
bioinfo_studsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_studsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE"), "Q3847505", bioinfo_studsel['NM_ENTIDADE_ENSINO']) #UFRN
bioinfo_studsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_studsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE FEDERAL DE MINAS GERAIS"), "Q132140", bioinfo_studsel['NM_ENTIDADE_ENSINO']) #UFMG
bioinfo_studsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_studsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE DE SÃO PAULO"), "Q835960", bioinfo_studsel['NM_ENTIDADE_ENSINO']) #USP
bioinfo_studsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_studsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE FEDERAL DO PARANÁ"), "Q1232831", bioinfo_studsel['NM_ENTIDADE_ENSINO']) #UFPR
bioinfo_studsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_studsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ"), "Q1131055", bioinfo_studsel['NM_ENTIDADE_ENSINO']) #UTFPR
bioinfo_studsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_studsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ (CORNÉLIO PROCÓPIO)"), "Q1131055", bioinfo_studsel['NM_ENTIDADE_ENSINO']) #UTFPR

In [122]:
df_stud = pd.DataFrame({"Name": [x.title() for x in bioinfo_studsel["NM_DISCENTE"].to_list()],
                        "FieldWork": "P101", 
                        "QID_Bioinfo": "Q128570",
                        "Affiliation": "S1416",
                        "QID_Program": bioinfo_studsel["CD_PROGRAMA_IES"], 
                        "Location": "S276",
                        "QID_University": bioinfo_studsel["NM_ENTIDADE_ENSINO"],
                        "LinkRef": "S854",
                        "Link": "https://dados.gov.br/dataset/coleta-de-dados-docentes-da-pos-graduacao-stricto-sensu-no-brasil-2017",
                        "Ref": "S248",
                        "QID_Capes": "Q113334264"})


worksheet_stud = sh.add_worksheet("Discentes", rows = "300", cols = "14")

worksheet_stud.update_cell(1, 1, df_stud.columns[0])
worksheet_stud.update_cell(1, 2, "QID_Name")
worksheet_stud.update_cell(1, 3, df_stud.columns[1])
worksheet_stud.update_cell(1, 4, df_stud.columns[2])
worksheet_stud.update_cell(1, 5, df_stud.columns[3])
worksheet_stud.update_cell(1, 6, df_stud.columns[4])
worksheet_stud.update_cell(1, 7, df_stud.columns[5])
worksheet_stud.update_cell(1, 8, df_stud.columns[6])
worksheet_stud.update_cell(1, 9, df_stud.columns[7])
worksheet_stud.update_cell(1, 10, df_stud.columns[8])
worksheet_stud.update_cell(1, 11, df_stud.columns[9])
worksheet_stud.update_cell(1, 12, df_stud.columns[10])

for i, row in df_stud.reset_index(drop = True).iterrows():
  worksheet_stud.update_cell(i+2, 1, row[0])
  worksheet_stud.update_cell(i+2, 2, "=WIKIDATASEARCH(A"+ str(i+2)+")")
  worksheet_stud.update_cell(i+2, 3, row[1])
  worksheet_stud.update_cell(i+2, 4, row[2])
  worksheet_stud.update_cell(i+2, 5, row[3])
  worksheet_stud.update_cell(i+2, 6, row[4])
  worksheet_stud.update_cell(i+2, 7, row[5])
  worksheet_stud.update_cell(i+2, 8, row[6])
  worksheet_stud.update_cell(i+2, 9, row[7])
  worksheet_stud.update_cell(i+2, 10, row[8])
  worksheet_stud.update_cell(i+2, 11, row[9])
  worksheet_stud.update_cell(i+2, 12, row[10])
  sleep(2)

###**Docentes Bioinfo**

Seleção de docentes que estão ligados a um programa de PG em Bioinfo. 


**Seleção de Informações**

*   Universidade
*   Nacionalidade
*   Código Programa
*   Última Titulação
*   Origem

In [26]:
dataprof = pd.read_csv(csv_links.get("prof"), sep = ";", encoding = "ISO-8859-1")

bioinfo_prof = dataprof[dataprof["NM_PROGRAMA_IES"].str.contains("BIOINFO")]

In [117]:
select_info = ["NM_DOCENTE","NM_ENTIDADE_ENSINO","DS_TIPO_NACIONALIDADE_DOCENTE","CD_PROGRAMA_IES","NM_AREA_BASICA_TITULACAO", "NM_IES_TITULACAO"]
bioinfo_profsel = bioinfo_prof.loc[:, select_info]


bioinfo_profsel

,NM_DOCENTE,NM_ENTIDADE_ENSINO,DS_TIPO_NACIONALIDADE_DOCENTE,CD_PROGRAMA_IES,NM_AREA_BASICA_TITULACAO,NM_IES_TITULACAO
11870,SIDARTA TOLLENDAL GOMES RIBEIRO,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,NEUROLOGIA,ROCKEFELLER UNIVERSITY
11917,RODRIGO JULIANI SIQUEIRA DALMOLIN,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,BIOQUÍMICA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE
11966,DANIEL CARLOS FERREIRA LANZA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,BIOQUÍMICA,UNIVERSIDADE ESTADUAL DE CAMPINAS
11982,JOSE MIGUEL ORTEGA,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,BIOQUÍMICA,UNIVERSIDADE DE SÃO PAULO
12114,RENAN CIPRIANO MOIOLI,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,BRASILEIRO,23001011170P8,CIÊNCIA DA COMPUTAÇÃO,UNIVERSITY OF SUSSEX
...,...,...,...,...,...,...
82298,FRANCISMAR CORREA MARCELINO GUIMARAES,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ (CO...,BRASILEIRO,40001016175P8,GENÉTICA,UNIVERSIDADE FEDERAL DE VIÇOSA
82302,DANILO SIPOLI SANCHES,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ (CO...,BRASILEIRO,40001016175P8,ENGENHARIA ELÉTRICA,UNIVERSIDADE DE SÃO PAULO (SÃO CARLOS)
82306,PRISCILA TIEMI MAEDA SAITO,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ (CO...,BRASILEIRO,40001016175P8,CIÊNCIA DA COMPUTAÇÃO,UNIVERSIDADE ESTADUAL DE CAMPINAS
82308,ANDRE YOSHIAKI KASHIWABARA,UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ (CO...,BRASILEIRO,40001016175P8,CIÊNCIA DA COMPUTAÇÃO,UNIVERSIDADE DE SÃO PAULO


In [118]:
bioinfo_profsel["CD_PROGRAMA_IES"] = np.where((bioinfo_profsel['CD_PROGRAMA_IES'] == "23001011170P8"), "Q105627005", bioinfo_profsel["CD_PROGRAMA_IES"]) #UFRN
bioinfo_profsel["CD_PROGRAMA_IES"] = np.where((bioinfo_profsel['CD_PROGRAMA_IES'] == "32001010068P4"), "Q105481707", bioinfo_profsel["CD_PROGRAMA_IES"]) #UFMG
bioinfo_profsel["CD_PROGRAMA_IES"] = np.where((bioinfo_profsel['CD_PROGRAMA_IES'] == "33002010188P9"), "Q102292035", bioinfo_profsel["CD_PROGRAMA_IES"]) #USP
bioinfo_profsel["CD_PROGRAMA_IES"] = np.where((bioinfo_profsel['CD_PROGRAMA_IES'] == "40001016066P4"),"Q105621954", bioinfo_profsel["CD_PROGRAMA_IES"]) #UFPR
bioinfo_profsel["CD_PROGRAMA_IES"] = np.where((bioinfo_profsel['CD_PROGRAMA_IES'] == "40001016175P8") & (bioinfo_profsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE FEDERAL DO PARANÁ"), "Q110097471", bioinfo_profsel["CD_PROGRAMA_IES"]) #UFPR
bioinfo_profsel["CD_PROGRAMA_IES"] = np.where((bioinfo_profsel['CD_PROGRAMA_IES'] == "40001016175P8") & (bioinfo_profsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ (CORNÉLIO PROCÓPIO)"), "Q110097471", bioinfo_profsel["CD_PROGRAMA_IES"]) #UTFPR
bioinfo_profsel["CD_PROGRAMA_IES"] = np.where((bioinfo_profsel['CD_PROGRAMA_IES'] == "40006018037P6"), "Q105627310", bioinfo_profsel["CD_PROGRAMA_IES"]) #UTFPR


print(bioinfo_profsel["CD_PROGRAMA_IES"])


11870    Q105627005
11917    Q105627005
11966    Q105627005
11982    Q105627005
12114    Q105627005
            ...    
82298    Q110097471
82302    Q110097471
82306    Q110097471
82308    Q110097471
82314    Q110097471
Name: CD_PROGRAMA_IES, Length: 146, dtype: object


In [119]:
bioinfo_profsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_profsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE"), "Q3847505", bioinfo_profsel['NM_ENTIDADE_ENSINO']) #UFRN
bioinfo_profsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_profsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE FEDERAL DE MINAS GERAIS"), "Q132140", bioinfo_profsel['NM_ENTIDADE_ENSINO']) #UFMG
bioinfo_profsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_profsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE DE SÃO PAULO"), "Q835960", bioinfo_profsel['NM_ENTIDADE_ENSINO']) #USP
bioinfo_profsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_profsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE FEDERAL DO PARANÁ"), "Q1232831", bioinfo_profsel['NM_ENTIDADE_ENSINO']) #UFPR
bioinfo_profsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_profsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ"), "Q1131055", bioinfo_profsel['NM_ENTIDADE_ENSINO']) #UTFPR
bioinfo_profsel['NM_ENTIDADE_ENSINO'] = np.where((bioinfo_profsel['NM_ENTIDADE_ENSINO'] == "UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ (CORNÉLIO PROCÓPIO)"), "Q1131055", bioinfo_profsel['NM_ENTIDADE_ENSINO']) #UTFPR



In [120]:
df_prof = pd.DataFrame({"Name": [x.title() for x in bioinfo_profsel["NM_DOCENTE"].to_list()],
                        "FieldWork": "P101", 
                        "QID_Bioinfo": "Q128570",
                        "Affiliation": "S1416",
                        "QID_Program": bioinfo_profsel["CD_PROGRAMA_IES"], 
                        "Location": "S276",
                        "QID_University": bioinfo_profsel["NM_ENTIDADE_ENSINO"],
                        "LinkRef": "S854",
                        "Link": "https://dados.gov.br/dataset/coleta-de-dados-docentes-da-pos-graduacao-stricto-sensu-no-brasil-2017",
                        "Ref": "S248",
                        "QID_Capes": "Q113334264"})


worksheet_prof = sh.add_worksheet("Docentes", rows = "300", cols = "14")

worksheet_prof.update_cell(1, 1, df_prof.columns[0])
worksheet_prof.update_cell(1, 2, "QID_Name")
worksheet_prof.update_cell(1, 3, df_prof.columns[1])
worksheet_prof.update_cell(1, 4, df_prof.columns[2])
worksheet_prof.update_cell(1, 5, df_prof.columns[3])
worksheet_prof.update_cell(1, 6, df_prof.columns[4])
worksheet_prof.update_cell(1, 7, df_prof.columns[5])
worksheet_prof.update_cell(1, 8, df_prof.columns[6])
worksheet_prof.update_cell(1, 9, df_prof.columns[7])
worksheet_prof.update_cell(1, 10, df_prof.columns[8])
worksheet_prof.update_cell(1, 11, df_prof.columns[9])
worksheet_prof.update_cell(1, 12, df_prof.columns[10])

for i, row in df_prof.reset_index(drop = True).iterrows():
  worksheet_prof.update_cell(i+2, 1, row[0])
  worksheet_prof.update_cell(i+2, 2, "=WIKIDATASEARCH(A"+ str(i+2)+")")
  worksheet_prof.update_cell(i+2, 3, row[1])
  worksheet_prof.update_cell(i+2, 4, row[2])
  worksheet_prof.update_cell(i+2, 5, row[3])
  worksheet_prof.update_cell(i+2, 6, row[4])
  worksheet_prof.update_cell(i+2, 7, row[5])
  worksheet_prof.update_cell(i+2, 8, row[6])
  worksheet_prof.update_cell(i+2, 9, row[7])
  worksheet_prof.update_cell(i+2, 10, row[8])
  worksheet_prof.update_cell(i+2, 11, row[9])
  worksheet_prof.update_cell(i+2, 12, row[10])
  sleep(2)
